<a href="https://colab.research.google.com/github/zairaamatori/data_driven_insights/blob/main/petroleo_gas_eda_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ingestão de Dados(integração com a tabela do bigquery)

In [ ]:
!pip install --upgrade google-cloud-bigquery  # instalando a biblioteca do bigquery


In [ ]:
# autenticação de usuário
from google.colab import auth
auth.authenticate_user()

In [ ]:
# importando a biblioteca do bigquery
from google.cloud import bigquery

# importando as bibliotecas do python
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

idProjeto = "t1engenhariadados" # id do projeto no bigquery

cliente = bigquery.Client(project = idProjeto) # criando um cliente que chama o projeto que contém a tabela do bigquery

In [ ]:
# Criando uma query que seleciona toda as linhas e colunas da tabela do bigquery, criando e lendo um dataframe
query = """

SELECT * FROM `t1engenhariadados.projeto_final_3_3.petroleo_gas`;

"""

resultado = cliente.query(query)
df = resultado.to_dataframe()
df.head()


In [ ]:
# Verifica a quantidade de linhas e colunas e imprime
df.shape
linhas, colunas = df.shape[0] , df.shape[1]
print(f'Linhas - {linhas}')
print(f'Colunas - {colunas}')

In [ ]:
# Conta  e mostra o número de linhas duplicadas (se houver)
df.duplicated().sum()
df[df.duplicated()]

In [ ]:
# Converter col 'Production Date' para o formato de data (datetime),
# permitindo extrair info como ano, mês, etc.
df['Production Date'] = pd.to_datetime(df['Production Date'])
# Criar uma coluna 'Ano' e 'Mes' para facilitar o agrupamento por períodos determinados de tempo
df['Ano'] = df['Production Date'].dt.year
df['Mes'] = df['Production Date'].dt.month

Criação de coluna padronizada de volume

In [ ]:
df['VolumeBOE'] = np.where(df['Commodity'] == 'Oil (bbl)', df['Volume'], df['Volume'] / 6)
# formatação sem exponencial
pd.options.display.float_format = '{:.2f}'.format

Classificando volume produzido e volume de residuos enviados

In [ ]:
# criando nova coluna para classificação de volume
df['Tipo'] = np.where(df['Volume']>0, 'Produzido', 'Residuo')


In [ ]:
# Criar colunas separadas por commodity
df['VolumeGas'] = np.where(df['Commodity'] == 'Gas (Mcf)', df['VolumeBOE'],0)
df['VolumeOil'] = np.where(df['Commodity'] == 'Oil (bbl)', df['VolumeBOE'],0)


In [ ]:
# Criar colunas separadas por Tipo
df['Volume_Produzido'] = np.where(df['Tipo'] == 'Produzido', df['VolumeBOE'],0)
df['Volume_Residuo'] = np.where(df['Tipo'] == 'Residuo', df['VolumeBOE'],0)


In [ ]:
# Criar colunas separadas por Tipo (Produzido / Resíduo)
df['VolumeGas_Produzido'] = np.where(['Volume_Produzido'] & (df['VolumeGas'].notna()),df['VolumeGas'],0)
df['VolumeGas_Residuo'] = np.where(['Volume_Residuo'] & (df['VolumeGas'].notna()),df['VolumeGas'],0)
df['VolumeOil_Produzido'] = np.where(['Volume_Produzido'] & (df['VolumeOil'].notna()),df['VolumeOil'],0)
df['VolumeOil_Residuo'] = np.where(['Volume_Residuo'] & (df['VolumeOil'].notna()),df['VolumeOil'],0)


In [ ]:
# Visualização inicial dos dados
df.info()
df[['VolumeGas_Produzido', 'VolumeGas_Residuo', 'VolumeOil_Produzido', 'VolumeOil_Residuo', 'Volume_Produzido', 'Volume_Residuo']].describe()

In [ ]:
# Mostra a contagem de valores =0 por coluna
print((df == 0).sum())

In [ ]:
#Contagens de zeros por colunas por Ano
contagem_por_ano = (df[['VolumeGas_Produzido',
                       'VolumeGas_Residuo', 'VolumeOil_Produzido', 'VolumeOil_Residuo', 'VolumeBOE']] == 0).groupby(df['Ano']).sum()
print(contagem_por_ano)

In [ ]:

estatisticas_com_filtro_total = df[df['Volume'] != 0][['VolumeGas_Produzido', 'VolumeGas_Residuo','VolumeOil_Produzido', 'VolumeOil_Residuo', 'Volume_Produzido', 'Volume_Residuo']].describe()

print(estatisticas_com_filtro_total)

In [ ]:
# Mostra os valores únicos em determinada dimensão
df[df.isnull().any(axis=1)]

EXPLORAÇÃO

Disposition Description por Tipo

In [ ]:
 # O groupby agora é feito apenas pela descrição
aggregated_df = df.groupby(['Disposition Description']).agg({
    'Volume_Produzido': 'sum',
    'Volume_Residuo': 'sum'
}).reset_index().sort_values(
    by=['Volume_Produzido', 'Volume_Residuo'],
    ascending=[False, True]
)

print("Dados agregados e ordenados que serão plotados:")
print(aggregated_df)




In [ ]:
# ============================================================================
# 10. TOP 10 TIPOS DE DISPOSIÇÃO
# ============================================================================


disposicao = df.groupby('Disposition Description').agg({
    'Volume_Produzido': 'sum',
    'Volume_Residuo': lambda x: abs(x.sum())
}).reset_index()

disposicao['Volume_Total'] = disposicao['Volume_Produzido'] + disposicao['Volume_Residuo']
disposicao = disposicao.sort_values('Volume_Total', ascending=True).tail(10)

fig, ax = plt.subplots(figsize=(12, 8))
bars = ax.barh(range(len(disposicao)), disposicao['Volume_Total'],
               color='#9B59B6', alpha=0.8, edgecolor='black')

ax.set_yticks(range(len(disposicao)))
ax.set_yticklabels(disposicao['Disposition Description'], fontsize=9)
ax.set_xlabel('Volume Total (BOE)', fontsize=12)
ax.set_title('Top 10 Tipos de Disposição por Volume', fontsize=14, fontweight='bold')
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x/1e9:.1f}B'))
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('10_top10_disposicao.png', dpi=300, bbox_inches='tight')

In [ ]:
print('--- Volume por tipo ---')
print(f"Volume Total Produzido: {df['Volume_Produzido'].sum()}")
print(f"Volume Total de Resíduo: {df['Volume_Residuo'].sum()}")

# Gráfico

labels = ['Volume Produzido', 'Volume Resíduo']
valores = [df['Volume_Produzido'].sum(),df['Volume_Residuo'].sum()]
cores = ['#4A90E2', '#E2725B'] # Azul para produzido, Laranja/Salmão para resíduo

#Cria o gráfico de barras
plt.figure(figsize=(16, 8)) # Define o tamanho da figura (opcional)
barras = plt.bar(labels, valores, color=cores)

# Adiciona títulos e rótulos
plt.ylabel('Volume')
plt.title('Comparativo entre Volume Produzido e Volume de Resíduo')

# 6. Exibe o gráfico
plt.show()


In [ ]:
# --- Volume por Classificação ---
vol_oil_prod = df['VolumeOil_Produzido'].sum()
vol_oil_res = df['VolumeOil_Residuo'].sum()
vol_gas_prod = df['VolumeGas_Produzido'].sum()
vol_gas_res = df['VolumeGas_Residuo'].sum()

print('--- Volume por Classificação ---')
print(f"Volume de Óleo Produzido: {vol_oil_prod}")
print(f"Volume de Óleo Resíduo: {vol_oil_res}")
print(f"Volume de Gás Produzido: {vol_gas_prod}")
print(f"Volume de Gás Resíduo: {vol_gas_res}")


# Gráfico
labels = ['Óleo Produzido', 'Óleo Resíduo', 'Gás Produzido', 'Gás Resíduo']
valores = [vol_oil_prod, vol_oil_res, vol_gas_prod, vol_gas_res]
# Usando azul para produção e laranja/vermelho para resíduo
cores = ['#0d47a1', '#64b5f6', '#0039cb', '#90caf9']

# Cria o gráfico de barras
plt.figure(figsize=(16, 8))
barras = plt.bar(labels, valores, color=cores)

# Adiciona títulos e rótulos
plt.ylabel('Volume')
plt.title('Volume por Tipo e Categoria (Óleo e Gás)')

# Melhora a legibilidade dos rótulos do eixo x se forem longos
plt.xticks(rotation=15, ha="right")
plt.tight_layout() # Ajusta o gráfico para caber tudo

# Exibe o gráfico
plt.savefig('classificacao_volume.png')




Análise Disposition Description

In [ ]:
print(df['Disposition Description'].value_counts())

**Análise Classes de Terreno**

In [ ]:
# Contagem exata de cada classe
print("--- Contagem Absoluta ---")
print(df['Land Class'].value_counts())

print("\n--- Porcentagem do Total ---")
# Mostra a proporção de cada classe (normalizado)
print(df['Land Class'].value_counts(normalize=True) * 100)

In [ ]:
df['Land Class'].value_counts().plot(kind='bar',figsize =(16, 8))
plt.yticks(fontsize=10)
plt.xticks(fontsize=10, rotation=45, ha='right')
plt.title('Distribuição de Classes de Solo', fontsize=12)
plt.xlabel('Classe de Solo', fontsize=12)
plt.ylabel('Frequência', fontsize=12)
plt.tight_layout()
plt.show()


existem 3 tipos de classes de terrenos:
Federal, Mixed Exploratory e Native American

**Análise Classes de Terreno**

In [ ]:
# Contagem exata de cada classe
print("--- Contagem Absoluta ---")
print(df['Land Category'].value_counts())

print("\n--- Porcentagem do Total ---")
# Mostra a proporção de cada classe (normalizado)
print(df['Land Category'].value_counts(normalize=True) * 100)

In [ ]:
offshore_onshore = df.groupby(['Land Category', 'Commodity'])['Volume_Produzido'].sum().reset_index()
offshore_pivot = offshore_onshore.pivot(index='Land Category', columns='Commodity', values='Volume_Produzido')

fig, ax = plt.subplots(figsize=(10, 6))
offshore_pivot.plot(kind='bar', ax=ax, color=['#FF6B6B', '#4ECDC4'], alpha=0.8, edgecolor='black', width=0.7)
ax.set_xlabel('Categoria de Terreno', fontsize=12)
ax.set_ylabel('Volume Produzido (BOE)', fontsize=12)
ax.set_title('Produção Offshore vs Onshore por Commodity', fontsize=14, fontweight='bold')
ax.legend(title='Commodity', fontsize=10)
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x/1e9:.1f}B'))
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('05_offshore_onshore.png', dpi=300, bbox_inches='tight')

Análise referente as regiões marítimas onde ocorre produção de recursos

In [ ]:
# Contagem exata de cada classe
print("--- Contagem Absoluta ---")
print(df['Offshore Region'].value_counts())

print("\n--- Porcentagem do Total ---")
# Mostra a proporção de cada classe (normalizado)
print(df['Offshore Region'].value_counts(normalize=True) * 100)

Estados e Condados

In [ ]:
df['State'].nunique()
df['State'].value_counts()

In [ ]:
df['County'].nunique()
df['County'].value_counts()

Recursos extraídos

In [ ]:
# Contagem exata de cada classe
print("--- Contagem Absoluta ---")
quant = df['Commodity'].value_counts()
print(quant)

print("\n--- Porcentagem do Total ---")
# Mostra a proporção de cada classe (normalizado)
porcen = df['Commodity'].value_counts(normalize=True) * 100
print(porcen)

A produção de gás e óleo foi distribuída de forma igualitária, tendo sido metade de óleo e metade de gás.

In [ ]:
analise_ocorrencias = df.groupby(['Ano', 'Commodity']).size()
print(analise_ocorrencias)

In [ ]:
analise_volume = df.groupby(['Ano', 'Commodity'])['Volume_Produzido'].sum()
print(analise_volume)
# soma as quantidades

Foi produzido 39.969.934.160 mais de gás do que de óleo.

In [ ]:
analise_volume.unstack().plot(kind='bar', figsize=(16, 8), color=['#03045e', '#00b4d8'])
plt.xlabel('Ano',fontsize=10)
plt.yticks(fontsize=10 )
plt.xticks(fontsize=10)
plt.ylabel('Volume Produzido ',fontsize=10)
plt.title('Produção de Recursos Anual',fontsize=10)

Quantidade de recursos produzida por mês

In [ ]:
# Lista com os nomes dos meses
nomes_meses = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']

# Agrupamento por mês
dados_grafico = df.groupby('Mes')['Volume_Produzido'].sum().sort_index()

# Gráfico
plt.figure(figsize=(16, 8))
plt.plot(dados_grafico.index, dados_grafico.values, marker='o')
plt.xticks(ticks=range(1, 13), labels=nomes_meses, fontsize=12)
plt.xlabel('Mês', fontsize=12)
plt.ylabel('Volume Produzido', fontsize=12)
plt.title('Quantidade Produzida por Mês', fontsize=12)
plt.yticks(fontsize=12)
plt.grid(False)
plt.tight_layout()
plt.show()


Perfil de produção para cada Classe territorial


In [ ]:
analise = df.groupby(['Land Class', 'Commodity'])['Volume_Produzido'].sum()
print(analise)

In [ ]:
analise.sort_values(ascending=False).unstack().plot(kind='barh', figsize=(16, 8), color=['#0077b6', '#00b4d8'])
plt.yticks(fontsize=10 )
plt.xticks(fontsize=10)
plt.xlabel('Classe de Solo', fontsize=12)
plt.ylabel('Volume Produzido', fontsize=12)
plt.title('Perfil de Produção por Classe de Solo', fontsize=12)


Produção entre condados e estados

In [ ]:
on_off = df.groupby(['State', 'County'])['Volume'].sum()
print(on_off)

In [ ]:
df.groupby(['Commodity', 'State', 'Ano'])['Volume'].sum()

Ano e Estado com maior produção de Gas

In [ ]:
df_gas = df[df['Commodity'] == 'Gas (Mcf)']
gas_production = df.groupby(['Ano', 'State'])['VolumeGas_Produzido'].sum().reset_index()
max_gas_production = gas_production.loc[gas_production['VolumeGas_Produzido'].idxmax()]
print(max_gas_production)


Ano e Estado com Maior produção de Oléo

In [ ]:
oil_production = df.groupby(['Ano', 'State'])['VolumeOil_Produzido'].sum().reset_index()
max_oil_production = oil_production.loc[oil_production['VolumeOil_Produzido'].idxmax()]
print(max_oil_production)

Ano e Estado com menor produção de Oléo

In [ ]:
oil_production = df.groupby(['Ano', 'State'])['VolumeOil_Produzido'].sum().reset_index()
min_oil_production = oil_production.loc[oil_production['VolumeOil_Produzido'].idxmin()]
print(min_oil_production)

Participação percentual de cada commodity na produção total de cada ano

In [ ]:
# Agrupar por Ano e Commodity e somar os volumes
# Isso cria um novo DataFrame com o volume total para cada commodity em cada ano.
comm_ano = df.groupby(['Ano', 'Commodity'])['Volume_Produzido'].sum().reset_index()

# Passo 2: Calcular o volume total para cada ano (agora no DataFrame agregado)
comm_ano['total_volume_ano'] = comm_ano.groupby('Ano')['Volume_Produzido'].transform('sum')

# Passo 3: Calcular a participação percentual correta
comm_ano['percentual'] = (comm_ano['Volume_Produzido'] / comm_ano['total_volume_ano']) * 100

# Passo 4: Formatar e limpar para o resultado final
comm_ano['percentual'] = comm_ano['percentual'].map('{:.2f}%'.format)
comm_ano = comm_ano.drop(columns=['total_volume_ano'])

# Exibe o DataFrame final com o insight correto
print("Participação Percentual TOTAL de Cada Commodity por Ano:")
print(comm_ano)

In [ ]:
# Agrupar por ano e somar os volumes produzidos
producao_anual = df.groupby('Ano')[['VolumeGas_Produzido', 'VolumeOil_Produzido']].sum()

# --- Passo 3: Geração do Gráfico ---

# Configurações do gráfico
anos = producao_anual.index
gas_volume = producao_anual['VolumeGas_Produzido']
oil_volume = producao_anual['VolumeOil_Produzido']

x = np.arange(len(anos))  # Posições dos anos no eixo X
width = 0.4  # Largura das barras

# Criar a figura e os eixos
fig, ax = plt.subplots(figsize=(16, 8))

# Plotar as barras de Gás e Óleo
rects1 = ax.bar(x - width/2, gas_volume, width, label='Gas (Mcf)', color='#45c4b8', edgecolor='black')
rects2 = ax.bar(x + width/2, oil_volume, width, label='Oil (bbl)', color='#ff8c8c', edgecolor='black')

# Adicionar títulos e rótulos
ax.set_title('Comparação de Produção: Gás vs Óleo (2015-2025)', fontsize=18)
ax.set_xlabel('Ano', fontsize=14)
ax.set_ylabel('Volume Produzido (BOE)', fontsize=14)

# Configurar os ticks do eixo X
ax.set_xticks(x)
ax.set_xticklabels(anos)
ax.tick_params(axis='x', labelsize=12)
ax.tick_params(axis='y', labelsize=12)



# Adicionar legenda e grade de fundo
ax.legend(fontsize=12)
ax.grid(True, which='major', axis='y', linestyle='-', linewidth=0.5, color='gray', alpha=0.3)
ax.set_axisbelow(True) # Coloca a grade atrás das barras

# Otimizar layout para evitar sobreposição
fig.tight_layout()

# Mostrar o gráfico
plt.show()

print("\nGráfico gerado com sucesso!")

Taxa de Desperdicio

In [ ]:
#Taxa de desperdício
print("3. Gerando gráfico de taxa de desperdício...")

eficiencia_temporal = df.groupby('Ano').agg({
    'Volume_Produzido': 'sum',
    'Volume_Residuo': lambda x: abs(x.sum())
}).reset_index()

eficiencia_temporal['Taxa_Desperdicio'] = (
    eficiencia_temporal['Volume_Residuo'] /
    (eficiencia_temporal['Volume_Produzido'] + eficiencia_temporal['Volume_Residuo'])
) * 100

fig, ax = plt.subplots(figsize=(12, 6))
bars = ax.bar(eficiencia_temporal['Ano'], eficiencia_temporal['Taxa_Desperdicio'],
               color='#E74C3C', alpha=0.7, edgecolor='black')

# Adicionar valores nas barras
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.2f}%', ha='center', va='bottom', fontsize=9)

ax.set_xlabel('Ano', fontsize=12)
ax.set_ylabel('Taxa de Desperdício (%)', fontsize=12)
ax.set_title('Evolução da Taxa de Desperdício (2015-2025)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('03_taxa_desperdicio_temporal.png', dpi=300, bbox_inches='tight')


### Média Anual de produção de oléo e gás

In [ ]:

# 6. SAZONALIDADE DA PRODUÇÃO

print("6. Gerando gráfico de sazonalidade...")

sazonalidade = df.groupby(['Mes', 'Commodity'])['Volume_Produzido'].mean().reset_index()
sazonalidade_pivot = sazonalidade.pivot(index='Mes', columns='Commodity', values='Volume_Produzido')

meses = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']

fig, ax = plt.subplots(figsize=(12, 6))
for commodity in sazonalidade_pivot.columns:
    ax.plot(sazonalidade_pivot.index, sazonalidade_pivot[commodity],
            marker='o', linewidth=2, markersize=8, label=commodity)

ax.set_xlabel('Mês', fontsize=12)
ax.set_ylabel('Volume Médio Produzido', fontsize=12)
ax.set_title('Padrão de Sazonalidade da Produção', fontsize=14, fontweight='bold')
ax.set_xticks(range(1, 13))
ax.set_xticklabels(meses)
ax.legend(title='Commodity', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('06_sazonalidade.png', dpi=300, bbox_inches='tight')

In [ ]:
# ============================================================================
# 1. DISTRIBUIÇÃO DE VOLUMES (LOG SCALE)
# ============================================================================
print("1. Gerando gráfico de distribuição de volumes...")

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gás
df_gas_prod = df[(df['Commodity'] == 'Gas (Mcf)') & (df['Volume_Produzido'] > 0)]
axes[0].hist(np.log10(df_gas_prod['Volume_Produzido']), bins=50, color='skyblue', edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Log10(Volume Produzido)', fontsize=11)
axes[0].set_ylabel('Frequência', fontsize=11)
axes[0].set_title('Distribuição de Volume de Gás (Escala Logarítmica)', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Óleo
df_oil_prod = df[(df['Commodity'] == 'Oil (bbl)') & (df['Volume_Produzido'] > 0)]
axes[1].hist(np.log10(df_oil_prod['Volume_Produzido']), bins=50, color='coral', edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Log10(Volume Produzido)', fontsize=11)
axes[1].set_ylabel('Frequência', fontsize=11)
axes[1].set_title('Distribuição de Volume de Óleo (Escala Logarítmica)', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('01_distribuicao_volumes.png', dpi=300, bbox_inches='tight')



In [ ]:
# Salvando em csv
df.to_csv('petroleo_gas_tratados.csv', index=False)

In [ ]:
# Baixar em csv
from google.colab import files
files.download('petroleo_gas_tratados.csv')